# Рекомендация тарифов:

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings('ignore')

## Открытие и изучение файла:

In [2]:
data = pd.read_csv('D://Education//Module_5 (!Project)//users_behavior.csv')

### **Основная информация датафрейма (метода info()):**

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


### **Первые 10 строк датафрейма:**

In [4]:
display(data.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### **Первичное описание имеющихся данных:**

**сalls** — количество звонков,  
**minutes** — суммарная длительность звонков в минутах,  
**messages** — количество sms-сообщений,  
**mb_used** — израсходованный интернет-трафик в Мб,  
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

### **Проверка на наличие пропусков данных и дубликотов в столбцах:**

In [5]:
print(data.isna().sum())

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


In [6]:
print(data.duplicated().sum())

0


## Разбивка данных на выборки:

### **Деление датафрейма:**

Разделим выборку в соответствии с долями:
- 60% - тренировочная,
- 20% - валидационная,
- 20% - тестировочная. 

In [7]:
data_train, data_temp = train_test_split(data, test_size=0.40, random_state=12345)
data_valid, data_test = train_test_split(data_temp, test_size=0.50, random_state=12345)

In [8]:
print('Тренеровочная:', data_train.shape[0])
print('Валидационная:', data_valid.shape[0])
print('Тестовая:', data_test.shape[0])

Тренеровочная: 1928
Валидационная: 643
Тестовая: 643


### **Выделение признаков:**

Выделим признаки и целевые признаки для каждого датафрейма:

In [9]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

Данные разделены по принципу 60/20/20:  
- для обучения модели (data_train),
- для проверки модели (data_valid),
- для оценочного теста модели (data_test).

## Исследование модели:

В данном проекте решается задача классификации, следовательно, использоваться будут следующие модели:
- DecisionTreeClassifier (решающее дерево),
- RandomForestClassifier (случайный лес),
- LogisticRegression (логистическая регрессия).

### **Решающее дерево:**

In [10]:
best_depth = 0
best_result = 0
for depth in range(1, 51):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_depth = depth
        best_result = result

print('Лучшая глубина дерева:', best_depth)
print('Лучшая точность модели:', best_result)

Лучшая глубина дерева: 3
Лучшая точность модели: 0.7853810264385692


Наилучший результат при глубине дерева "3":

**Исследуем изменение результатов при смене гиперпараметров:**

*Изменим критерий с 'gini' на 'entropy':*

In [11]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3, criterion='entropy')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Точность:', result)

Точность: 0.7853810264385692


Точность при изменении параметра не изменилась.

*Изменим параметр 'splitter' с 'best' на 'random':*

In [12]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3, splitter='random')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Точность:', result)

Точность: 0.7387247278382582


Точность при изменении параметра уменьшилась.

*Изменение остальных гиперпараметров не привело к улучшению точности.*

**Таким образом, наилучший результат показывает решающее дерево с глубиной "3".**

In [13]:
best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=3)
best_tree_model.fit(features_train, target_train)
predictions_valid = best_tree_model.predict(features_valid)
tree_result = accuracy_score(target_valid, predictions_valid)
print('Точность:', tree_result)

Точность: 0.7853810264385692


### **Случайный лес:**

In [14]:
best_estimators = 0
best_result = 0

for est in range(1, 51):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_estimators = est
        best_result = result

print('Лучшее кол-во деревьев:', best_estimators)
print('Лучшая точность модели:', best_result)

Лучшее кол-во деревьев: 23
Лучшая точность модели: 0.7947122861586314


Наилучший результат при кол-ве деревьев "23":

**Исследуем изменение результатов при смене гиперпараметров:**

*Изменим критерий с 'gini' на 'entropy':*

In [15]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, criterion='entropy')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
forest_result = accuracy_score(target_valid, predictions_valid)
print('Точность:', forest_result)

Точность: 0.7838258164852255


Точность при изменении параметра уменьшилась.

*Изменим параметр 'warm_start' с 'False' на 'True':*

In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, warm_start='True')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
forest_result = accuracy_score(target_valid, predictions_valid)
print('Точность:', forest_result)

Точность: 0.7947122861586314


Точность при изменении параметра не изменилась.

*Изменим параметр 'bootstrap' с 'True' на 'False':*

In [17]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, bootstrap='False')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
forest_result = accuracy_score(target_valid, predictions_valid)
print('Точность:', forest_result)

Точность: 0.7947122861586314


Точность при изменении параметра не изменилась.

*Изменение остальных гиперпараметров не привело к улучшению точности.*

**Таким образом, наилучший результат показывает случайный лес с кол-вом деревьев "23".**

In [18]:
best_forest_model = RandomForestClassifier(random_state=12345, n_estimators=23)
best_forest_model.fit(features_train, target_train)
predictions_valid = best_forest_model.predict(features_valid)
forest_result = accuracy_score(target_valid, predictions_valid)
print('Точность:', forest_result)

Точность: 0.7947122861586314


### **Логистическая регрессия:**

In [19]:
best_iter = 0
best_result = 0

for num_iter in range(1, 1001):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=num_iter)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_iter = num_iter
        best_result = result

print('Лучшее кол-во итерраций:', best_iter)
print('Лучшая точность модели:', best_result) 

Лучшее кол-во итерраций: 39
Лучшая точность модели: 0.7107309486780715


**Исследуем изменение результатов при смене гиперпараметров:**

*Изменим параметр 'solver':*

In [20]:
model = LogisticRegression(random_state=12345, solver='newton-cg', max_iter=39)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Точность:', result)

Точность: 0.7573872472783826


In [21]:
model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=39)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Точность:', result)

Точность: 0.7076205287713841


In [22]:
model = LogisticRegression(random_state=12345, solver='sag', max_iter=39)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Точность:', result)

Точность: 0.7060653188180405


In [23]:
model = LogisticRegression(random_state=12345, solver='saga', max_iter=39)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Точность:', result)

Точность: 0.7060653188180405


Лучший показатель точности при параметре 'solver' = 'newton-cg'

*Изменение остальных гиперпараметров не привело к улучшению точности.*

**Таким образом, наилучший результат показывает логистическая регрессия с 39 итерациями и параметром solver = 'newton-cg'**

In [24]:
best_log_regres_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=39)
best_log_regres_model.fit(features_train, target_train)
predictions_valid = best_log_regres_model.predict(features_valid)
log_regres_result = accuracy_score(target_valid, predictions_valid)
print('Точность:', log_regres_result)

Точность: 0.7107309486780715


**В результате проведенного анализа установлено:**
- Наилучшая точность модели решающего дерева равна 0.7853810264385692 при глубине: 3.
- Наилучшая точность модели случайного леса равна 0.7947122861586314 при кол-ве деревьев: 23.
- Наилучшая точность модели логистической регрессии равна 0.7573872472783826 при параметре solver='newton-cg' и кол-ве итераций: 39.
  
**В данном исследовании лучшей моделью является модель случайного леса.**

## Проверка модели на тестовой выборке:

Для оценки полученных моделей проверим их точность с использование тестовой выборки:

**Для решающего дерева:**

In [25]:
tree_test_predictions = best_tree_model.predict(features_test)
result = accuracy_score(target_test, tree_test_predictions)
print('Точность решающего дерева:', result)

Точность решающего дерева: 0.7791601866251944


Точность решающего дерева:
- на валидационной выборке: 0.7853810264385692
- на тестовой выборке: 0.7791601866251944
- отклонение: 0.0062208398133748

**Для случайного леса:**

In [26]:
forest_test_predictions = best_forest_model.predict(features_test)
result = accuracy_score(target_test, forest_test_predictions)
print('Точность случайного леса:', result)

Точность случайного леса: 0.7807153965785381


Точность случайного леса:
- на валидационной выборке: 0.7947122861586314
- на тестовой выборке: 0.7807153965785381
- отклонение: 0.0139968895800933

**Для логистической регрессии:**

In [27]:
log_regres_test_predictions = best_log_regres_model.predict(features_test)
result = accuracy_score(target_test, log_regres_test_predictions)
print('Точность случайного леса:', result)

Точность случайного леса: 0.6842923794712286


Точность логистической регрессии:
- на валидационной выборке: 0.7573872472783826
- на тестовой выборке: 0.6842923794712286
- отклонение: 0.073094867807154

Лучший показатель точности у случайного леса.  
При этом, наименьшее отклонение значений результатов точности между валидационной и тестовой выборками имеет модель решающего дерева, что свидетельствует о минимальной переобученности модели.
Наихудший результат показала модель логистической регрессии.

Так как гиперпараметры подобраны - валидационная выборка более не нужна. Объединим тренировочную (data_train) и валидационную (data_valid) выборки, с целью обучения модели на большей выборке.


In [28]:
data_merge = pd.concat([data_train,data_valid], axis=0)

In [29]:
display(data_merge.head(5))

,calls,minutes,messages,mb_used,is_ultra
3027,60.0,431.56,26.0,14751.26,0
434,33.0,265.17,59.0,17398.02,0
1226,52.0,341.83,68.0,15462.38,0
1054,42.0,226.18,21.0,13243.48,0
1842,30.0,198.42,0.0,8189.53,0


Проведем тестирование модели на объединенной выборке (data_merge):

In [30]:
features_merge = data_merge.drop(['is_ultra'], axis=1)
target_merge = data_merge['is_ultra']

In [31]:
model = RandomForestClassifier(random_state=12345, n_estimators=23)
model.fit(features_merge, target_merge)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('Точность:', result)

Точность: 0.8055987558320373


Исходя из условий задачи, необходимо достигнуть уровня точности не менее 0,75.  
Таким образом, для решения поставленной задачи допустимо использовать как модель решающего дерева, так и модель случайного леса (целесообразнее модель случайного леса).

## Проверка модели на адекватность:

In [32]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_merge, target_merge)
dummy_pred = dummy.predict(features_test)

# checking unique labels
print('Unique predicted labels: ', (np.unique(dummy_pred)))

# checking accuracy
print('Test score: ', accuracy_score(target_test, dummy_pred))

Unique predicted labels:  [0]
Test score:  0.6842923794712286


Полученная модель имеет "среднее" качество.